In [20]:
#####################做出 price survey real
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")


df=pd.read_excel('每月更新表格.xlsx', sheet_name ='US',encoding='big5')
df['date'] = pd.to_datetime(df['date'],format='%Y-%m')
df.sort_values("date",inplace=True)
df=df.set_index(keys=['date'])
df=df['1994':]   #從1994的data開始

#變數YOY化
df['標普500指數YOY']=(df['標普500指數']/df['標普500指數'].shift(12))-1
df['美國非農業就業人口淨變動 經季調(月比)YOY']=(df['美國非農業就業人口淨變動 經季調(月比)']/df['美國非農業就業人口淨變動 經季調(月比)'].shift(12))-1
df['美國中小企業樂觀指數YOY']=(df['美國中小企業樂觀指數']/df['美國中小企業樂觀指數'].shift(12))-1
df['密西根大學消費者信心指數(未來指標)YOY']=(df['密西根大學消費者信心指數(未來指標)']/df['密西根大學消費者信心指數(未來指標)'].shift(12))-1
df['美國經濟諮詢委員會領先指標YOY']=(df['美國經濟諮詢委員會領先指標']/df['美國經濟諮詢委員會領先指標'].shift(12))-1
df['美國經濟諮詢委員會實質消費者信心指數 經季調YOY']=(df['美國經濟諮詢委員會實質消費者信心指數 經季調']/df['美國經濟諮詢委員會實質消費者信心指數 經季調'].shift(12))-1
df['Bloomberg West Texas Intermediate (WTI) Cushing Crude Oil Spot Price YOY']=(df['Bloomberg West Texas Intermediate (WTI) Cushing Crude Oil Spot Price']/df['Bloomberg West Texas Intermediate (WTI) Cushing Crude Oil Spot Price'].shift(12))-1
df['UST10YOY']=(df['UST10']/df['UST10'].shift(12))-1
#利率差分  (10y-3m)
df['3M10Y利差']=df['USGG10YR Index']-df['USGG3M Index']
#波動率反向化  
df['X']=-1
df['美林公債波動']=df['美林公債波動']*df['X']
df=df.drop('X', 1)



column_name=set(df.columns)  #轉成set
#30個月z-score化  
for variable in column_name: 
    df[variable]=(df[variable]-df[variable].rolling(30).mean())/df[variable].rolling(30).std()



#做出 price survey actual
df['price']=(df['US financial index']+df['標普500指數YOY']+df['Bloomberg West Texas Intermediate (WTI) Cushing Crude Oil Spot Price YOY']+df['Commodities index']+df['3M10Y利差']+df['UST10YOY']+df['美林公債波動'])/8                                            

df['survey']=(df['美國供應管理協會全國製造業採購經理人指數']+df['美國中小企業樂觀指數YOY']+df['美國NAHB住宅市場指數']+df['美國經濟諮詢委員會領先指標YOY']+df['密西根大學消費者信心指數(未來指標)YOY']+df['美國經濟諮詢委員會實質消費者信心指數 經季調YOY']+df['ZEW美國經濟成長預期']+df['ZEW美國經濟成長現況'])/10               

df['actual']=(df['美國 OECD']+df['美國非農業就業人口淨變動 經季調(月比)YOY']+df['工業生產(年比)']+df['美國消費者物價指數年比']+df['美國平均時薪(當時美元)(經季調)(年比)']+df['美國個人消費支出核心平減指數 經季調(年比)']+df['經濟成長'])/7


df=df.dropna()
df['月']=df.index.month


#計算上行&下行訊號
df['上行訊號']=0
df['下行訊號']=0
df['p+s']=df['price']+df['survey']
for i in range(0,len(df['上行訊號'])-1,1): 
        up1=(df['survey'].iloc[i]>df['survey'].iloc[i-1]) & (df['survey'].iloc[i-1]>df['survey'].iloc[i-2]) & (df['survey'].iloc[i-2]>df['survey'].iloc[i-3])  
        up2=(df['price'].iloc[i]>df['price'].iloc[i-1]) & (df['price'].iloc[i-1]>df['price'].iloc[i-2]) & (df['price'].iloc[i-2]>df['price'].iloc[i-3])
        up5=(df['p+s'].iloc[i]>df['p+s'].iloc[i-1]) & (df['p+s'].iloc[i-1]>df['p+s'].iloc[i-2]) & (df['p+s'].iloc[i-2]>df['p+s'].iloc[i-3]) 
        down1=(df['survey'].iloc[i]<df['survey'].iloc[i-1]) & (df['survey'].iloc[i-1]<df['survey'].iloc[i-2]) & (df['survey'].iloc[i-2]<df['survey'].iloc[i-3])  
        down2=(df['price'].iloc[i]<df['price'].iloc[i-1]) & (df['price'].iloc[i-1]<df['price'].iloc[i-2]) & (df['price'].iloc[i-2]<df['price'].iloc[i-3])
        down5=(df['p+s'].iloc[i]<df['p+s'].iloc[i-1]) & (df['p+s'].iloc[i-1]<df['p+s'].iloc[i-2]) & (df['p+s'].iloc[i-2]<df['p+s'].iloc[i-3])

        if (up1==True) & (up2==True)& (up5==True):    
            df['上行訊號'].iloc[i]=1
        if (down1==True) & (down2==True)& (down5==True):    
            df['下行訊號'].iloc[i]=-1   


rsp=df.loc[:, ['price' , 'survey','actual']]

rsp.corr()




C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,price,survey,actual
price,1.000000,0.628231,0.626866
survey,0.628231,1.000000,0.611615
actual,0.626866,0.611615,1.000000


In [21]:
#####################做出dashboard


rsp=df.loc[:, ['price' , 'survey','actual']]


#print('走勢圖', end='')
#rsp.plot()
#plt.show()


table=df.drop(columns=['月'])
table=table.rank(pct=True, ascending=False)*100



     



#變異來源board    以百分比計算
change=(df.diff()/df.shift()).iloc[-1]

price_group=df.loc[:,['price','US financial index','標普500指數YOY','Bloomberg West Texas Intermediate (WTI) Cushing Crude Oil Spot Price YOY','Commodities index','美元指數','3M10Y利差','UST10YOY','美林公債波動']]
survey_group=df.loc[:,['survey','美國供應管理協會全國製造業採購經理人指數','美國中小企業樂觀指數YOY','美國NAHB住宅市場指數','美國經濟諮詢委員會領先指標YOY','密西根大學消費者信心指數(未來指標)YOY','美國經濟諮詢委員會實質消費者信心指數 經季調YOY','ZEW美國經濟成長預期','ZEW美國經濟成長現況','IFO 全球景氣6個月預期','IFO 全球景氣現況']]
actual_group=df.loc[:,['actual','美國 OECD','美國非農業就業人口淨變動 經季調(月比)YOY','工業生產(年比)','美國消費者物價指數年比','美國平均時薪(當時美元)(經季調)(年比)','美國個人消費支出核心平減指數 經季調(年比)','經濟成長']]

price_group_change1=(price_group.diff()).iloc[-1]        #1個月
survey_group_change1=(survey_group.diff()).iloc[-1]
actual_group_change1=(actual_group.diff()).iloc[-1]

price_group_change3=(price_group.diff(3)).iloc[-1]        #3個月
survey_group_change3=(survey_group.diff(3)).iloc[-1]
actual_group_change3=(actual_group.diff(3)).iloc[-1]

price_group_change6=(price_group.diff(6)).iloc[-1]        #6個月
survey_group_change6=(survey_group.diff(6)).iloc[-1]
actual_group_change6=(actual_group.diff(6)).iloc[-1]

price_group_change12=(price_group.diff(12)).iloc[-1]        #12個月
survey_group_change12=(survey_group.diff(12)).iloc[-1]
actual_group_change12=(actual_group.diff(12)).iloc[-1]




#price_group_change1.loc['price']=(df['price']-df['price'].shift()).iloc[-1]    #修正total
#price_group_change3.loc['price']=(df['price']-df['price'].shift(3)).iloc[-1]
#price_group_change6.loc['price']=(df['price']-df['price'].shift(6)).iloc[-1]
#price_group_change12.loc['price']=(df['price']-df['price'].shift(12)).iloc[-1]
#
#survey_group_change1.loc['survey']=(df['survey']-df['survey'].shift()).iloc[-1]    #修正total
#survey_group_change3.loc['survey']=(df['survey']-df['survey'].shift(3)).iloc[-1]
#survey_group_change6.loc['survey']=(df['survey']-df['survey'].shift(6)).iloc[-1]
#survey_group_change12.loc['survey']=(df['survey']-df['survey'].shift(12)).iloc[-1]
#
#actual_group_change1.loc['actual']=(df['actual']-df['actual'].shift()).iloc[-1]    #修正total
#actual_group_change3.loc['actual']=(df['actual']-df['actual'].shift(3)).iloc[-1]
#actual_group_change6.loc['actual']=(df['actual']-df['actual'].shift(6)).iloc[-1]
#actual_group_change12.loc['actual']=(df['actual']-df['actual'].shift(12)).iloc[-1]






#產出所有 to excel
writer=pd.ExcelWriter('US_現況dashboard.xlsx')

table.iloc[-1].to_excel(writer,'現況分數')
df.to_excel(writer,'個變數歷史明細')
price_group_change1.to_excel(writer,'變異',startrow=1,startcol=0)
survey_group_change1.to_excel(writer,'變異',startrow=1,startcol=3)
actual_group_change1.to_excel(writer,'變異',startrow=1,startcol=6)

price_group_change3.to_excel(writer,'變異',startrow=18,startcol=0)
survey_group_change3.to_excel(writer,'變異',startrow=18,startcol=3)
actual_group_change3.to_excel(writer,'變異',startrow=18,startcol=6)

price_group_change6.to_excel(writer,'變異',startrow=35,startcol=0)
survey_group_change6.to_excel(writer,'變異',startrow=35,startcol=3)
actual_group_change6.to_excel(writer,'變異',startrow=35,startcol=6)

price_group_change12.to_excel(writer,'變異',startrow=50,startcol=0)
survey_group_change12.to_excel(writer,'變異',startrow=50,startcol=3)
actual_group_change12.to_excel(writer,'變異',startrow=50,startcol=6)


writer.save()


#在excel中貼上標示
from openpyxl import load_workbook

wb = load_workbook('US_現況dashboard.xlsx')
ws = wb['變異']
ws['A1'] = '1個月變異'
ws['A18'] = '3個月變異'
ws['A35'] = '6個月變異'
ws['A50'] = '12個月變異'
wb.save('US_現況dashboard.xlsx')



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [30]:
'''
#####################回測 底部survey+price 連2個月往上,   actual??
df['p+s']=df['price']+df['survey']
rsp=df.loc[:, ['price' , 'survey','actual','p+s']]

#rsp['actual+1']=rsp['actual'].shift(-1)-rsp['actual']
#rsp['actual+2']=rsp['actual'].shift(-2)-rsp['actual'].shift(-1)
#rsp['actual+3']=rsp['actual'].shift(-3)-rsp['actual'].shift(-2)
#rsp['actual+4']=rsp['actual'].shift(-4)-rsp['actual'].shift(-3)
#rsp['actual+5']=rsp['actual'].shift(-5)-rsp['actual'].shift(-4)
#rsp['actual+6']=rsp['actual'].shift(-6)-rsp['actual'].shift(-5)
#rsp['actual+7']=rsp['actual'].shift(-7)-rsp['actual'].shift(-6)
#rsp['actual+8']=rsp['actual'].shift(-8)-rsp['actual'].shift(-7)
#rsp['actual+9']=rsp['actual'].shift(-9)-rsp['actual'].shift(-8)
#rsp['actual+10']=rsp['actual'].shift(-10)-rsp['actual'].shift(-9)
#rsp['actual+11']=rsp['actual'].shift(-11)-rsp['actual'].shift(-10)
#rsp['actual+12']=rsp['actual'].shift(-12)-rsp['actual'].shift(-11)

rsp['actual+6']=rsp['actual'].shift(-6)-rsp['actual']

con1=(rsp['survey']>rsp['survey'].shift()) & (rsp['survey'].shift()>rsp['survey'].shift(2)) & (rsp['survey'].shift(2)>rsp['survey'].shift(3))  
con2=(rsp['price']>rsp['price'].shift()) & (rsp['price'].shift()>rsp['price'].shift(2)) & (rsp['price'].shift(2)>rsp['price'].shift(3))
#con3=(rsp['survey']-rsp['survey'].shift(3))>0.1
#con4=(rsp['price']-rsp['price'].shift(3))>0.1
con5=(rsp['p+s']>rsp['p+s'].shift()) & (rsp['p+s'].shift()>rsp['p+s'].shift(2)) & (rsp['p+s'].shift(2)>rsp['p+s'].shift(3))

#con5=rsp['actual']<0
#print(rsp[con1&con2&con3&con4&con5])
backtest=rsp[con1&con2&con5]

backtest
#backtest.to_excel("底部survey+price 連2個月往上.xlsx")
'''

,price,survey,actual,p+s,actual+6
date,,,,,
1999-03-31,-0.015267,-0.648511,-0.549806,-0.663778,0.594035
1999-04-30,0.235933,-0.617650,-0.292720,-0.381717,0.588157
2001-06-29,0.098236,-0.807979,-1.273483,-0.709744,-0.213607
2002-01-31,-0.139279,-0.188069,-1.641103,-0.327348,0.930463
2002-02-28,-0.006194,-0.119704,-1.485594,-0.125898,0.953113
2002-03-29,0.261566,0.453506,-1.163758,0.715072,1.180883
2003-08-29,0.892250,0.742817,0.361868,1.635067,0.887105
2009-03-31,-0.559722,-1.002647,-2.407153,-1.562369,1.294264
2009-04-30,-0.386257,-0.411526,-2.100226,-0.797783,1.271966


In [33]:
'''
#####################回測 頭部survey+price 連2個月往下,   actual??

df['p+s']=df['price']+df['survey']
rsp=df.loc[:, ['price' , 'survey','actual','p+s']]

#rsp['actual+1']=rsp['actual'].shift(-1)-rsp['actual']
#rsp['actual+2']=rsp['actual'].shift(-2)-rsp['actual'].shift(-1)
#rsp['actual+3']=rsp['actual'].shift(-3)-rsp['actual'].shift(-2)
#rsp['actual+4']=rsp['actual'].shift(-4)-rsp['actual'].shift(-3)
#rsp['actual+5']=rsp['actual'].shift(-5)-rsp['actual'].shift(-4)
#rsp['actual+6']=rsp['actual'].shift(-6)-rsp['actual'].shift(-5)
#rsp['actual+7']=rsp['actual'].shift(-7)-rsp['actual'].shift(-6)
#rsp['actual+8']=rsp['actual'].shift(-8)-rsp['actual'].shift(-7)
#rsp['actual+9']=rsp['actual'].shift(-9)-rsp['actual'].shift(-8)
#rsp['actual+10']=rsp['actual'].shift(-10)-rsp['actual'].shift(-9)
#rsp['actual+11']=rsp['actual'].shift(-11)-rsp['actual'].shift(-10)
#rsp['actual+12']=rsp['actual'].shift(-12)-rsp['actual'].shift(-11)


rsp['actual+6']=rsp['actual'].shift(-6)-rsp['actual']

con1=(rsp['survey']<rsp['survey'].shift()) & (rsp['survey'].shift()<rsp['survey'].shift(2)) & (rsp['survey'].shift(2)<rsp['survey'].shift(3))  
con2=(rsp['price']<rsp['price'].shift()) & (rsp['price'].shift()<rsp['price'].shift(2)) & (rsp['price'].shift(2)<rsp['price'].shift(3))
#con3=(rsp['survey']-rsp['survey'].shift(3))<-0.1
#con4=(rsp['price']-rsp['price'].shift(3))<-0.1
con5=(rsp['p+s']<rsp['p+s'].shift()) & (rsp['p+s'].shift()<rsp['p+s'].shift(2)) & (rsp['p+s'].shift(2)<rsp['p+s'].shift(3))

#con5=rsp['actual']>0
#print(rsp[con1&con2&con3&con4&con5])
backtest=rsp[con1&con2&con5]

backtest
#backtest.to_excel("頭部survey+price 連2個月往下.xlsx")
'''

,price,survey,actual,p+s,actual+6
date,,,,,
1997-10-31,-0.310546,0.988288,0.447289,0.677741,-0.297568
1998-07-31,-0.585179,-0.482384,-0.270595,-1.067563,-0.290537
1998-08-31,-1.649995,-0.689988,-0.059495,-2.339983,-0.375855
2000-11-30,-0.270311,-0.811156,-0.781150,-1.081467,-0.448098
2000-12-29,-0.666628,-1.653031,-1.295207,-2.319660,0.021724
2010-08-31,-0.001417,0.484134,0.765701,0.482717,0.155443
2011-05-31,0.471116,0.284848,0.982256,0.755964,-0.328198
2011-06-30,0.418366,0.056208,0.957258,0.474574,-0.280970
2011-07-29,0.084799,-0.049796,0.956823,0.035003,-0.295295
